In [2]:
import sys
sys.path.append('C:/Users/sOrOush/SoroushProjects/01_Soroush_and_Shakiba/NSD_High_Dimensional_Data/11_Marco_And_Soroush/Scripts/GANalyze/pytorch')

In [3]:
import numpy as np
mean = np.load('C:/Users/sOrOush/SoroushProjects/01_Soroush_and_Shakiba/NSD_High_Dimensional_Data/11_Marco_And_Soroush/Data/GANalyze/pytorch/assessors/image_mean.npy')

In [7]:
from assessors import memnet


In [9]:
import os
import sys
import torch
import numpy as np
import socket
import argparse
import json
import subprocess
from contextlib import contextmanager
import torch.distributed as dist
from torch.nn.parallel.distributed import DistributedDataParallel
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as T
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt

sub = 1
bs = 30

In [10]:
assessor = memnet.MemNet()

C:\Users/sOrOush/SoroushProjects/01_Soroush_and_Shakiba/NSD_High_Dimensional_Data/11_Marco_And_Soroush/Scripts/GANalyze/pytorch\assessors\memnet.py:62: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  state_dict = pickle.load(f)


In [11]:
def test_images_prep_assessor_CLIP(images, target_size=(256, 256), mean=None):
    data_tensor = []

    # Define the transformation: resizing, channel reordering, mean subtraction, cropping, and converting to tensor
    transform = T.Compose([
        T.Resize(target_size),
        T.Lambda(lambda x: np.array(x)),  # Convert image to numpy array
        T.Lambda(lambda x: np.subtract(x[:,:,[2, 1, 0]], mean) if mean is not None else x),  # Subtract average mean from image (BGR order if mean is provided)
        T.Lambda(lambda x: x[15:242, 15:242]),  # Center crop
        T.ToTensor()  # Convert to tensor and scale to [0, 1]
    ])

    # Iterate over each in-memory image
    for im in images:
        # Apply the transformations (resize, mean subtraction, cropping, and to tensor)
        img_tensor = transform(im)
        data_tensor.append(img_tensor)

    # Stack the list of tensors into a single tensor
    return torch.stack(data_tensor)

In [12]:
# Transfor latents from flattened representation to hierarchical
def latent_transformation(latents, ref):
  layer_dims = np.array([2**4,2**4,2**8,2**8,2**8,2**8,2**10,2**10,2**10,2**10,2**10,2**10,2**10,2**10,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**14])
  transformed_latents = []
  for i in range(31):
    t_lat = latents[:,layer_dims[:i].sum():layer_dims[:i+1].sum()]
    #std_norm_test_latent = (t_lat - np.mean(t_lat,axis=0)) / np.std(t_lat,axis=0)
    #renorm_test_latent = std_norm_test_latent * np.std(kamitani_latents[i][num_test:].reshape(num_train,-1),axis=0) + np.mean(kamitani_latents[i][num_test:].reshape(num_train,-1),axis=0)
    c,h,w=ref[i]['z'].shape[1:]
    transformed_latents.append(t_lat.reshape(len(latents),c,h,w))
  return transformed_latents

# idx = range(len(test_images))
# input_latent = latent_transformation(pred_latents[idx],ref_latent)


def sample_from_hier_latents(latents,sample_ids):
  sample_ids = [id for id in sample_ids if id<len(latents[0])]
  layers_num=len(latents)
  sample_latents = []
  for i in range(layers_num):
    sample_latents.append(torch.tensor(latents[i][sample_ids]).float().cuda())
  return sample_latents


In [13]:
emonet_res = {
    'nsdgeneral_degree1': [],
    'nsdgeneral_degree2': [],
    'nsdgeneral_degree3': [],
    'nsdgeneral_degree4': [],
    'Default_degree1': [],
    'Default_degree2': [],
    'Default_degree3': [],
    'Default_degree4': [],
    'Auditory_degree1': [],
    'Auditory_degree2': [],
    'Auditory_degree3': [],
    'Auditory_degree4': []
}

In [14]:
## real_img
image_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\00_Original_Images"
test_tensor = test_images_prep_assessor_CLIP(image_path)
# Reshape while keeping the last 3 dimensions
test_im = test_tensor.view(-1, *test_tensor.shape[-3:])

#------  Apply the transformations and assess
score = assessor(test_im)
emonet_res['real_img'] = [s.detach().numpy()[0] for s in score]
print("====Real Done!====")


TypeError: Unexpected type <class 'str'>